## CPN,LVQ神经网络

### CPN实现函数逼近

In [2]:
import matplotlib.pyplot as plt
from PIL import Image
import os

class ImageSequenceGIF:
    def __init__(self, gif_dir='temp_gif_frames', gif_file='output.gif'):
        self.gif_dir = gif_dir
        self.gif_file = gif_file
        self.frames = []
        self.frame_count = 0

        if not os.path.exists(gif_dir):
            os.makedirs(gif_dir)

    def add_frame_from_plt(self, plt_figure=None):
        if plt_figure is None:
            plt_figure = plt.gcf()

        filename = os.path.join(self.gif_dir, f"frame_{self.frame_count:04d}.png")
        plt_figure.savefig(filename, bbox_inches='tight')
        self.frames.append(Image.open(filename))
        self.frame_count += 1
        plt.close(plt_figure)

    def save_gif(self, duration=100, last_frame_duration=200):
        if not self.frames:
            print("No frames to save.")
            return

        durations = [duration] * (len(self.frames) - 1) + [last_frame_duration]
        self.frames[0].save(
            self.gif_file,
            save_all=True,
            append_images=self.frames[1:],
            duration=durations,
            loop=0
        )
        print(f"GIF saved as {self.gif_file}")

    def clear_temp_frames(self):
        for f in os.listdir(self.gif_dir):
            os.remove(os.path.join(self.gif_dir, f))
        self.frames = []
        self.frame_count = 0


#### 单向CPN

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import exp, linspace, dot, amin, where, array, zeros, eye
from numpy.linalg import inv

# Hermite多项式函数
def f_hermit(x):
    return 1.1 * (1 - x + 2 * x**2) * exp(-x**2 / 2)

# 生成训练数据
TRAIN_DATA_NUM = 500
np.random.seed(0)
x_train = np.random.uniform(-4, 4, TRAIN_DATA_NUM)
y_train = f_hermit(x_train)

# 显示结果
def show_res(x, y):
    x_line = linspace(-4, 4, 250)
    plt.clf()
    plt.plot(x_line, f_hermit(x_line), '--', c='grey', linewidth=1, label='Hermit Func')
    plt.scatter(x, y, s=10, c='darkviolet', label='Train Data')
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.grid(True)
    plt.legend(loc='upper right')
    plt.tight_layout()

# 最近邻竞争函数
def WTA_nearest(x, v):
    err = [x - vv for vv in v]
    dist = [dot(e, e) for e in err]
    id = where(dist == amin(dist))[0][0]
    return id

# K近邻更新函数
def K_neighbor(v, x, eta):
    for xx in x:
        id = WTA_nearest(xx, v)
        v[id] = v[id] + eta * (xx - v[id])
    return v

# CPN输出函数
def CPN_v_out(x, v):
    H = []
    for xx in x:
        h = zeros(v.shape[0])
        id = WTA_nearest(xx, v)
        h[id] = 1
        H.append(list(h))
    return array(H).T

# CPN权重更新函数
def CPN_W(h, y):
    vv = h.T.dot(h) + 0.000001 * eye(h.shape[1])
    vvv = inv(vv).dot(h.T)
    return y.reshape(1, -1).dot(vvv)

# 设置参数
NODE_NUM = 20
v_data = x_train[0:NODE_NUM]  # 初始化神经元中心
ETA_BEGIN = 0.1
ETA_END = 0.0
TRAIN_STEP = 100

# 初始化GIF生成类
gif_maker = ImageSequenceGIF(gif_dir='temp_gif_frames', gif_file='GIF/cpn_single.gif')

# 训练过程
for id, eta in enumerate(np.linspace(ETA_BEGIN, ETA_END, TRAIN_STEP)):
    v_data = K_neighbor(v_data, x_train, eta)  # 更新神经元中心

    # 扩展训练数据
    x_train_ex = x_train
    y_train_ex = f_hermit(x_train_ex)

    # 计算隐层输出和权重更新
    H = CPN_v_out(x_train_ex, v_data)
    W = CPN_W(H, y_train_ex)
    yy = W.dot(H)

    # 绘制逼近曲线
    y_train = f_hermit(x_train)
    show_res(x_train, y_train)

    x_line = np.linspace(-4, 4, 500)
    Hx = CPN_v_out(x_line, v_data)
    y_line = W.dot(Hx)

    plt.plot(x_line, y_line[0], c='red', linewidth=1, label='Net Performance')
    v_H = CPN_v_out(v_data, v_data)
    v_yy = W.dot(v_H)
    plt.scatter(v_data, v_yy[0], s=30, c='darkcyan', label='V value')

    plt.title(f'Step: {id}, Eta: {eta:.2f}')
    plt.tight_layout()
    plt.legend(loc='upper right')
    plt.draw()
    gif_maker.add_frame_from_plt(plt.gcf())  # 将当前帧添加到 GIF 中

# 保存 GIF
gif_maker.save_gif(duration=100, last_frame_duration=500)
plt.show()


GIF saved as GIF/cpn_single.gif


![](GIF/cpn_single.gif)

#### 双向CPN

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import exp, linspace, dot, amin, where, array, zeros, eye
from numpy.linalg import inv
from PIL import Image
import os

# Hermite多项式函数
def f_hermit(x):
    return 1.1 * (1 - x + 2 * x**2) * np.exp(-x**2 / 2)

# 生成训练数据
TRAIN_DATA_NUM = 500
np.random.seed(0)
x_train = np.random.uniform(-4, 4, TRAIN_DATA_NUM)
y_train = f_hermit(x_train)

# 显示结果
def show_res(x, y):
    x_line = np.linspace(-4, 4, 250)
    plt.clf()
    plt.plot(x_line, f_hermit(x_line), '--', c='grey', linewidth=1, label='Hermit Func')
    plt.scatter(x, y, s=10, c='darkviolet', label='Train Data')
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.grid(True)
    plt.legend(loc='upper right')
    plt.tight_layout()

# 最近邻竞争函数
def WTA_nearest(x, v):
    err = [x - vv for vv in v]
    dist = [dot(e, e) for e in err]
    id = where(dist == amin(dist))[0][0]
    return id

# K近邻更新函数
def K_neighbor(v, x, eta):
    for xx in x:
        id = WTA_nearest(xx, v)
        v[id] = v[id] + eta * (xx - v[id])
    return v

# CPN输出函数
def CPN_v_out(x, v):
    H = []
    for xx in x:
        h = zeros(v.shape[0])
        id = WTA_nearest(xx, v)
        h[id] = 1
        H.append(list(h))
    return array(H).T

# CPN权重更新函数
def CPN_W(h, y):
    vv = h.T.dot(h) + 0.000001 * eye(h.shape[1])
    vvv = inv(vv).dot(h.T)
    return y.reshape(1, -1).dot(vvv)

# 设置参数
NODE_NUM = 20  # 设置隐层节点数量为20
v_data = x_train[0:NODE_NUM]  # 初始化神经元中心
ETA_BEGIN = 0.1
ETA_END = 0.0
TRAIN_STEP = 100

# 初始化GIF生成类
gif_maker = ImageSequenceGIF(gif_dir='temp_gif_frames', gif_file='GIF/cpn_double.gif')

# 训练过程
for id, eta in enumerate(np.linspace(ETA_BEGIN, ETA_END, TRAIN_STEP)):
    v_data = K_neighbor(v_data, x_train, eta)  # 更新神经元中心

    # 扩展训练数据
    x_train_ex = x_train
    y_train_ex = f_hermit(x_train_ex)

    # 计算隐层输出和权重更新
    H = CPN_v_out(x_train_ex, v_data)
    W = CPN_W(H, y_train_ex)
    yy = W.dot(H)

    # 绘制逼近曲线
    y_train = f_hermit(x_train)
    show_res(x_train, y_train)

    x_line = np.linspace(-4, 4, 500)
    Hx = CPN_v_out(x_line, v_data)
    y_line = W.dot(Hx)

    # 绘制网络逼近性能
    plt.plot(x_line, y_line[0], c='red', linewidth=1, label='Net Performance')
    v_H = CPN_v_out(v_data, v_data)
    v_yy = W.dot(v_H)
    
    # 显示隐层节点的值
    plt.scatter(v_data, v_yy[0], s=30, c='darkcyan', label='Hidden Node Value')
    plt.title(f'Step: {id}, Eta: {eta:.2f}')
    plt.tight_layout()
    plt.legend(loc='upper right')
    plt.draw()
    gif_maker.add_frame_from_plt(plt.gcf())  # 将当前帧添加到 GIF 中

# 保存 GIF
gif_maker.save_gif(duration=100, last_frame_duration=500)
plt.show()


GIF saved as GIF/cpn_double.gif


![](GIF/cpn_double.gif)

In [11]:
import numpy as np

# Hermite多项式函数
def f_hermit(x):
    return 1.1 * (1 - x + 2 * x**2) * np.exp(-x**2 / 2)

# 生成测试数据
TEST_DATA_NUM = 200
x_test = np.linspace(-4, 4, TEST_DATA_NUM)
y_test = f_hermit(x_test)

# 计算均方误差
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# 单向CPN和双向CPN的预测函数
def single_cpn_predict(x_train, y_train, x_test, hidden_nodes=20):
    v_data = x_train[:hidden_nodes]
    for eta in np.linspace(0.1, 0.0, 100):
        v_data = K_neighbor(v_data, x_train, eta)
    H = CPN_v_out(x_train, v_data)
    W = CPN_W(H, y_train)
    H_test = CPN_v_out(x_test, v_data)
    return W.dot(H_test)[0]

def double_cpn_predict(x_train, y_train, x_test, hidden_nodes=20):
    v_data = np.vstack((x_train[:hidden_nodes], f_hermit(x_train[:hidden_nodes]))).T
    xy_train = np.vstack((x_train, y_train)).T
    for eta in np.linspace(0.1, 0.0, 100):
        v_data = K_neighbor(v_data, xy_train, eta)
    H = CPN_v_out(x_train, v_data[:, 0])
    W = CPN_W(H, y_train)
    H_test = CPN_v_out(x_test, v_data[:, 0])
    return W.dot(H_test)[0]

# 执行单向和双向CPN的逼近，并计算精度
y_single_pred = single_cpn_predict(x_train, y_train, x_test)
y_double_pred = double_cpn_predict(x_train, y_train, x_test)

# 计算精度
mse_single = mse(y_test, y_single_pred)
mse_double = mse(y_test, y_double_pred)

print(f"Single CPN MSE: {mse_single:.5f}")
print(f"Double CPN MSE: {mse_double:.5f}")


Single CPN MSE: 0.02414
Double CPN MSE: 0.02269


#### 精度对比

* 单向CPN的MSE值：单向CPN仅使用输入值来训练隐层节点，对于逼近函数的能力有限，因此MSE值相对可能会高一些。
* 双向CPN的MSE值：双向CPN通过在隐层节点中同时考虑输入和输出信息，使得模型在训练过程中更准确地捕捉到数据的分布特性，因此MSE值可能会更低。

### LVQ网络分类

In [31]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import dot, array, amin, where
import random
from sklearn.datasets import make_circles

def gif1_data(datanum=300):
    x1 = np.linspace(-3, 3, datanum)
    noise1 = np.random.randn(datanum) * 0.15
    y1 = -np.square(x1) / 3 + 4.5 + noise1
    data1 = np.column_stack((x1, y1))

    x2 = np.linspace(0, 6, datanum)
    noise2 = np.random.randn(datanum) * 0.15
    y2 = np.square(x2 - 3) / 3 + 0.5 + noise2
    data2 = np.column_stack((x2, y2))

    data = np.vstack((data1, data2))
    x = data[:, 0:2]
    y = np.array([0] * datanum + [1] * datanum)
    return x, y

# def gif1_data(datanum=300, scale=2.0):
#     x, y = make_circles(n_samples=datanum, noise=0.07, factor=0.6, random_state=random.randint(0, 1000))
#     x = x.astype(np.float64) * scale
#     # y = y.astype(np.float64) * scale
#     return x, y


# 生成数据集
x_train, y_train = gif1_data(300)

# 显示训练数据
def show_train_data(x, y):
    class_1 = x[y == 0]
    class_2 = x[y == 1]
    plt.scatter(class_1[:, 0], class_1[:, 1], s=10, c='blue', label='Class 1')
    plt.scatter(class_2[:, 0], class_2[:, 1], s=10, c='red', label='Class 2')

# 初始化权重，聚集在数据范围附近
def initialize_weights(num_neurons_per_class, data_dim):
    weights = []
    # 类别1节点分布在数据1附近，类别2节点分布在数据2附近
    for i in range(2):  # 两个类别
        if i == 0:
            x_range = (-3, 3)
            y_range = (4.5, 5.5)
        else:
            x_range = (0, 5)
            y_range = (0, 1)
        
        # 初始化每个类别的节点，使其位于对应的数据范围内
        weights_class = np.array([[random.uniform(*x_range), random.uniform(*y_range)] for _ in range(num_neurons_per_class)])
        weights.append(weights_class)
        
    return np.vstack(weights)

# # LVQ网络权重初始化函数
# def initialize_weights(num_neurons_per_class, data_dim):
#     weights = []
#     for _ in range(2):  # 两个类别
#         weights_class = array([[random.uniform(-3, 3), random.uniform(0, 6)] for _ in range(num_neurons_per_class)])
#         weights.append(weights_class)
#     return np.vstack(weights)

# 最近邻选择
def nearest_prototype(x, weights):
    distances = [dot(x - w, x - w) for w in weights]
    winner_index = np.argmin(distances)
    class_label = winner_index // (len(weights) // 2)  # 根据神经元个数判断类别
    return winner_index, class_label

# 动态调整学习率的LVQ训练函数
def lvq_train(x, y, weights, learning_rate, epoch, total_epochs):
    adjusted_lr = learning_rate * (1 - epoch / total_epochs)  # 动态调整学习率
    for i in np.random.permutation(len(x)):
        sample, true_class = x[i], y[i]
        winner_index, winner_class = nearest_prototype(sample, weights)
        
        # 更新权重
        if winner_class == true_class:
            weights[winner_index] += adjusted_lr * (sample - weights[winner_index])
        else:
            weights[winner_index] -= adjusted_lr * (sample - weights[winner_index])
    return weights

# 显示权重
def show_weights(weights, n_per_class):
    plt.axis([-4, 8, -2, 6])  # 更新可视化范围
    plt.scatter(weights[:n_per_class, 0], weights[:n_per_class, 1], s=75, c='darkblue', label='Node-Class1')
    plt.scatter(weights[n_per_class:, 0], weights[n_per_class:, 1], s=75, c='darkred', label='Node-Class2')
    show_train_data(x_train, y_train)
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.grid(True)
    plt.legend(loc="upper left")
    plt.tight_layout()

# 主函数 - 训练和测试LVQ网络
def run_lvq(num_neurons_per_class):
    data_dim = x_train.shape[1]
    weights = initialize_weights(num_neurons_per_class, data_dim)

    eta_start, eta_end, steps = 0.2, 0.01, 100  # 提高初始学习率
    learning_rates = np.linspace(eta_start, eta_end, steps)
    gif_maker = ImageSequenceGIF(gif_dir='temp_gif_frames', gif_file=f'GIF/lvq_{num_neurons_per_class}nodes.gif')

    for epoch, eta in enumerate(learning_rates):
        weights = lvq_train(x_train, y_train, weights, eta, epoch, steps)
        
        # 可视化每次迭代的权重
        plt.clf()
        show_weights(weights, num_neurons_per_class)
        plt.title(f'Step: {epoch+1}/{steps}, Learning Rate: {eta:.4f}')
        plt.draw()
        gif_maker.add_frame_from_plt(plt.gcf())
    
    gif_maker.save_gif(duration=100, last_frame_duration=500)
    gif_maker.clear_temp_frames()
    plt.show()

# 运行两种神经元数量的测试
print("LVQ with 6 neurons per class")
run_lvq(3)

print("LVQ with 20 neurons per class")
run_lvq(10)


LVQ with 6 neurons per class
GIF saved as GIF/lvq_3nodes.gif
LVQ with 20 neurons per class
GIF saved as GIF/lvq_10nodes.gif


六个神经元


![](GIF/lvq_3nodes.gif)

20个神经元

![](GIF/lvq_10nodes.gif)